### Apache Spark: распределенное исполнение

![alt text](static/spark_components.JPG "components")

Высокоуровнево Spark приложение состоит из программы-драйвера ("driver") 
ответственной за "оркестрацию" параллельных операций на кластера. Драйвер имеет доступ к распределенным компонентам кластера, таким как Spark Executors и cluster manager, через SparkSession. 

<p><b>Spark Driver</b></p>
Отвественен за поднятие SparkSession. Драйвер взаимодействует с кластер менеджером, запрашивая вычислительные ресурсы для исполнителей, трансформирует Spark операции в DAG вычисления, планирует их, распределяет задачи (Task) по всем исполнителям. Как только ресурсы выделены, общается напрямую с исполнителями

<p><b>Spark Session</b></p>
<p>С версии 2.0 стал единой точкой доступа к всей функциональности спарка, задаем параметры запуска JVM, доступ к метаданным каталога, определение DataFrame и прочее.
</p>
### Распределенные данные и партиции

![alt text](static/logical_model.JPG "model")

<p>Данные распределены по хранилищу в виде партиций, живущих в HDFS или облачном хранилище. В то время как данные распределены по физическому кластеру, Спарк обращается с каждой партицией как с выскоуровневой абстракцией (DataFrame в памяти). Каждый исполнитель предпочтительно выделяет задачу, требующую чтения ближайшей партиции.</p>

![alt text](static/exec_partitions.JPG "exec_part")

Разбиение ("партиционирование") позволяет обрабатывать задачи параллельно. Распределенная схема разбиения данных в блоки делает возможным обработку исполнителем ближайшей части, минимизируя нагрузку на сеть. Каждое ядро исполнителя ответственно за работу над одной партицией

### Spark application: основные понятия

![alt text](static/job.JPG "job")

Во время интерактивный сессии, драйвер преобразует спарк приложение в одну или несколько заданий (job). Затем каждое задание трансформируется в DAG. В сущности, это один план выполнения, где каждый узел (node) внутри DAG может состоять из одного или более этапов (stages).

![alt text](static/stages.JPG "stages")

Как часть узла, этапы образуются по принципу разделения операций на последовательные и параллельные. Часто этапы делятся по границам операторов, предполагающих передачу данных среди исполнителей.

![alt text](static/tasks.JPG "tasks")

Каждый этап состоит из задач (tasks), единиц исполнения, которые затем объединяются по исполнителям. Каждой задаче ставится в соответствие одно ядро, которое работает над одной партицией.

### Преобразования, действия и ленивые вычисления

Операции в спарке делятся на преобразования (transformations) и действия (actions). Преобразования преобразуют DataFrame в новый DataFrame без изменения исходных данных, гарантируя свойство неизменности. Например, операции select() или filter() не меняют исходный набор данных, а вместо этого возвращает преобразованный результат в виде нового DataFrame.

Все трансформации выолняются лениво, т.е результат не выполняется немедленно, а запоминается последовательность действий в виде "lineage". Записанная цепочка ,позже, позволяет спарку ,на этапе планирования, изменить определенные преобразования (комбинируя их различным образом, меняя порядок исполнения) для наиболее эффективного плана.

![alt text](static/lazy.JPG "lazy")

Стратегия ленивого исполнения откладывает вычисление до "пинка", операции действия (например, запись или чтение с диска). Действие затронет всю цепочку преобразований до своего появления. Важное свойств, устойчивость к отказу, записывая всю цепочку преобразований и гарантируя неизменность наборов данных, несложно восстановить потеренные вычисления

![alt text](static/tr_act.JPG "transform_action")

### Узкие и широкие преобразования

Узкие преобразования - любые преобразования, в которых одна выходная партиция может быть вычислена из одной входной. Например, filter() или contains(), они не требуют обмена данными. В то время как groupBy() или orderBy() являются примерами широких преобразований, данные из разных партиций считываются, объединяются и записываются на диск. Подсчет count() после groupBy() требует перетасовки данных от каждой партиции исполнителей во всем кластере.

![alt text](static/nar_wide.JPG "narrow_wide")

### DataFrame vs Dataset 

Концептуально, можно думать о DataFrame как о коллекции обобщенных объектов, Dataset[Row], где Row это generic нетипизированный JVM объект, способный содержать поля различных типов. Dataset ,напротив, коллекция строготипизированных типов (например, создаем класс и теперь Dataset хранит в себе объекты одного типа). Различие в том, что в DF мы использовали DSL операции, не зависящие от используемого языка, в то время как в DS можем использовать родные выражения Scala и Java.

<ul>
    <li>Когда хотим описать, что достать, а не как [DS, DF]</li>
    <li>Если хотим защищенность типов (compile type safety) берем DS</li>
    <li>Если хотим получить выигрыш от Tungsten сериализаторы [DS]</li>
    <li>Хотим унификации, простоты и оптимизации берем DF</li>
    <li>Обращаемся к RDD, в случае, когда мы знаем лучше</li>
</ul>

### Spark SQL и его движок

![alt text](static/spark_sql.JPG "spark_sql")

Задачи Spark Engine:
<ul>
    <li>Унификация компонентов и предоставляет абстракции DF и DS</li>
    <li>Соединение с Apache Hive</li>
    <li>Предоставление Spark SQL Shell для исследования данных</li>
    <li>Доступ к внешним СУБД через коннекторы</li>
    <li>Генерация оптимизированного плана запроса и кода для JVM</li>
</ul>

### Оптимизатор Catalyst

![alt text](static/catalyst.JPG "catalyst")

Catalyst представлен 4 фазами (Анализа, Логической оптимизации, Физического планнирования и генерации кода)

![alt text](static/catalyst_parsed.JPG "catalyst_parsed")

<p><b>Анализ</b> - на стадии анализа генерируется Абстрактное Синтаксическое дерево для SQL запроса. Обращаемся к каталогу для проверки колонок и названия таблички, в случае успеха идем на следующий этап</p>

<p><b>Логическая оптимизация</b> - состоит из двух этапов, на начальном этапе применяем оптимизацию на основе правил, так Catalyst построит множество планов на основе стоимостной оптимизации (Cost-based optimization) выберет лучший (по умполчанию CBO отключен). На этом этапе делаем predicate pushdown, projection prunning</p>

<p><b>Физическое планирование</b> - из оптимального логического на основе стратегий создаем физический план</p>

<p><b>Генерация кода</b> - на основе ФП, а именно физических операторов, с помощью проекта Tungsten и его замечательного кодогенератора (WholeStageCodeGen) создаем компактный RDD код</p>